In [201]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score


In [202]:
# wget https://raw.githubusercontent.com/alexeygrigorev/dfsets/master/course_lead_scoring.csv

In [203]:
df =pd.read_csv('course_lead_scoring.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [204]:
df.dtypes                        

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [205]:
df.columns


Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

# EDA

In [206]:
numerical_columns = ['number_of_courses_viewed','annual_income','interaction_count','lead_score']
categorical_columns = ['lead_source', 'industry', 'employment_status', 'location']

In [207]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [208]:
df[categorical_columns].nunique()

lead_source          5
industry             7
employment_status    4
location             7
dtype: int64

### df preparation<b>
Check if the missing values are presented in the features.<br>
If there are missing values:<br>
For caterogiral features, replace them with 'NA'<br>
For numerical features, replace with with 0.0<br>

In [209]:
df[categorical_columns].head()

,lead_source,industry,employment_status,location
0,paid_ads,NaN,unemployed,south_america
1,social_media,retail,employed,south_america
2,events,healthcare,unemployed,australia
3,paid_ads,retail,NaN,australia
4,referral,education,self_employed,europe


In [210]:
df[categorical_columns] = df[categorical_columns].fillna('NA')
df[numerical_columns] = df[numerical_columns].fillna(0)

In [211]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [212]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [213]:
# Question 1
# What is the most frequent observation (mode) for the column industry?

# NA
# technology
# healthcare
# retail

In [214]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

## Correlation Matrix

In [215]:
# Question 2
# Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

# What are the two features that have the biggest correlation?

# interaction_count and lead_score
# number_of_courses_viewed and lead_score
# number_of_courses_viewed and interaction_count
# annual_income and interaction_count
# Only consider the pairs above when answering this question.

In [216]:
print(df.interaction_count.corr(df.lead_score))
print(df.lead_score.corr(df.number_of_courses_viewed))
print(df.interaction_count.corr(df.number_of_courses_viewed))
print(df.annual_income.corr(df.interaction_count))

0.009888182496913077
-0.00487899835468127
-0.023565222882888107
0.027036472404814337


In [217]:
corr_matrix = df[numerical_columns].corr()
print(corr_matrix)

                          number_of_courses_viewed  annual_income  \
number_of_courses_viewed                  1.000000       0.009770   
annual_income                             0.009770       1.000000   
interaction_count                        -0.023565       0.027036   
lead_score                               -0.004879       0.015610   

                          interaction_count  lead_score  
number_of_courses_viewed          -0.023565   -0.004879  
annual_income                      0.027036    0.015610  
interaction_count                  1.000000    0.009888  
lead_score                         0.009888    1.000000  


In [218]:
corr_matrix.unstack().sort_values(ascending=False).drop_duplicates()

number_of_courses_viewed  number_of_courses_viewed    1.000000
annual_income             interaction_count           0.027036
lead_score                annual_income               0.015610
                          interaction_count           0.009888
annual_income             number_of_courses_viewed    0.009770
lead_score                number_of_courses_viewed   -0.004879
number_of_courses_viewed  interaction_count          -0.023565
dtype: float64

### Split the data

In [219]:
# Split your data in train/val/test sets with 60%/20%/20% distribution.
# Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
# Make sure that the target value y is not in your dataframe.

In [220]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [221]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [222]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [223]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [224]:
# Question 3
# Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
# Round the scores to 2 decimals using round(score, 2).
# Which of these variables has the biggest mutual information score?

# industry
# location
# lead_source
# employment_status

In [225]:
def mutual_info_converted_score(series):
    return mutual_info_score(series, y_train)

In [226]:
mi = df_train[categorical_columns].apply(mutual_info_converted_score)
mi.sort_values(ascending=False)

lead_source          0.035396
employment_status    0.012938
industry             0.011575
location             0.004464
dtype: float64

### One Hot Encoding

In [227]:
# Question 4
# Now let's train a logistic regression.
# Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
# Fit the model on the training dataset.
# To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
# model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
# Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
# What accuracy did you get?

# 0.64
# 0.74
# 0.84
# 0.94

In [228]:
from sklearn.feature_extraction import DictVectorizer

In [229]:
df_train[categorical_columns].nunique()

lead_source          6
industry             8
employment_status    5
location             8
dtype: int64

In [230]:
df_train.head(10)

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score
0,paid_ads,retail,0,58472.0,student,middle_east,5,0.03
1,organic_search,manufacturing,3,71738.0,student,middle_east,6,0.77
2,paid_ads,technology,3,81973.0,employed,north_america,2,0.59
3,NA,technology,1,74956.0,employed,europe,3,0.34
4,organic_search,retail,3,59335.0,student,australia,1,0.98
5,social_media,retail,4,50961.0,student,south_america,1,1.00
6,social_media,finance,0,0.0,unemployed,africa,3,0.73
7,events,retail,3,45170.0,student,middle_east,3,0.38
8,organic_search,NA,0,73717.0,employed,asia,2,0.28
9,organic_search,education,2,61872.0,student,australia,1,0.01


In [231]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical_columns + numerical_columns].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical_columns + numerical_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)

## 3.10 Training logistic regression with Scikit-Learn

In [232]:
from sklearn.linear_model import LogisticRegression

In [233]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [234]:
print(model.intercept_[0])
print(model.coef_[0].round(3))

-0.06914728027832363
[-0.    -0.015  0.034  0.003  0.012 -0.103 -0.025  0.049 -0.02  -0.013
 -0.003 -0.009 -0.032 -0.016  0.311  0.051  0.02  -0.012 -0.012 -0.115
  0.08  -0.03   0.004 -0.011 -0.011 -0.006  0.008  0.006 -0.033 -0.025
  0.454]


In [235]:
print(model.intercept_[0])
print(model.coef_[0].round(3))

-0.06914728027832363
[-0.    -0.015  0.034  0.003  0.012 -0.103 -0.025  0.049 -0.02  -0.013
 -0.003 -0.009 -0.032 -0.016  0.311  0.051  0.02  -0.012 -0.012 -0.115
  0.08  -0.03   0.004 -0.011 -0.011 -0.006  0.008  0.006 -0.033 -0.025
  0.454]


In [236]:
0.6996587030716723
0.7

0.7

In [237]:
# Calculate the accuracy on the validation dataset and round it to 2 decimal digits
y_pred = model.predict_proba(X_val)[:, 1]
converted = (y_pred >= 0.5)
# conv = model.predict(X_val)
actual_accuracy = (y_val == converted).mean()
print(actual_accuracy)
# print((y_val==conv).mean())
print(round(actual_accuracy, 2))

0.6996587030716723
0.7


## feature elimination technique

In [241]:
# Question 5
# Let's find the least useful feature using the feature elimination technique.
# Train a model using the same features and parameters as in Q4 (without rounding).
# Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
# For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
# Which of following feature has the smallest difference?

# 'industry'
# 'employment_status'
# 'lead_score'
# Note: The difference doesn't have to be positive.

In [242]:

def OneHotEncoding(df_train, df_val, col_name=None):
        
    dv = DictVectorizer(sparse=False)
    if(col_name):
        df_train_temp = df_train.drop(col_name, axis=1)
        df_val_temp = df_val.drop(col_name, axis=1)
    else:
        df_train_temp = df_train.copy()
        df_val_temp = df_val.copy()


    train_dict = df_train_temp.to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val_temp.to_dict(orient='records')
    X_val = dv.transform(val_dict)
    # print(df_train_temp.head(2))
    return X_train, X_val

In [243]:
col_tobe_removed = ['industry', 'employment_status', 'lead_score']
for c in col_tobe_removed:
    X_train, X_val = OneHotEncoding(df_train[categorical_columns + numerical_columns], df_val[categorical_columns + numerical_columns], c)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    converted = (y_pred >= 0.5)
    accuracy = (y_val == converted).mean()
    diff = actual_accuracy - accuracy
    print(f'Difference between the original accuracy and the accuracy without the feature {c} is {diff}')
    print()

Difference between the original accuracy and the accuracy without the feature industry is 0.0

Difference between the original accuracy and the accuracy without the feature employment_status is 0.0034129692832763903

Difference between the original accuracy and the accuracy without the feature lead_score is -0.0068259385665528916



In [244]:
# Question 6
# Now let's train a regularized logistic regression.
# Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
# Train models using all the features as in Q4.
# Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
# Which of these C leads to the best accuracy on the validation set?

# 0.01
# 0.1
# 1
# 10

In [247]:
C = [0.01, 0.1, 1, 10, 100]
X_train, X_val = OneHotEncoding(df_train[categorical_columns + numerical_columns], df_val[categorical_columns + numerical_columns])
for c in C:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    converted = (y_pred >= 0.5)
    accuracy = round((y_val == converted).mean(), 8)
    print(f'Accuracy score for c value {c} -->{accuracy}')

Accuracy score for c value 0.01 -->0.6996587
Accuracy score for c value 0.1 -->0.6996587
Accuracy score for c value 1 -->0.6996587
Accuracy score for c value 10 -->0.6996587
Accuracy score for c value 100 -->0.6996587


In [249]:
results = {}

for c_val in C:
    model = LogisticRegression(solver='liblinear', C=c_val, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy = round((y_val == y_pred).mean(), 3)
    results[c_val] = accuracy
    print(f"C={c_val} --> Accuracy={accuracy}")

# Best C
best_C = max(results, key=results.get)
print(f"\nBest C value: {best_C} with accuracy {results[best_C]}")

C=0.01 --> Accuracy=0.7
C=0.1 --> Accuracy=0.7
C=1 --> Accuracy=0.7
C=10 --> Accuracy=0.7
C=100 --> Accuracy=0.7

Best C value: 0.01 with accuracy 0.7
